#### install neccessary packages: 

os\
dotenv\
langchain_openai 
langchain_core.embeddings\
langchain.vectorstores\
langchain.docstore \
langchain.schema\
numpy (np)\
sklearn.metrics.pairwise

PyMuPDF\
langchain.document_loaders\
langchain.text_splitter\
faiss

example: !pip install os

###  Import Libraries

## Libraries Overview:

1. **os**: A standard library that allows interaction with the operating system, such as environment variable access.

2. **dotenv**: Used to load environment variables from a `.env` file. This is useful for managing sensitive information like API keys.

3. **AzureOpenAI**: This is the library used to interact with the Azure OpenAI service. It helps you communicate with the Azure-hosted OpenAI models, such as `GPT-4o` and embeddings models.

4. **langchain.document_loaders.PyPDFLoader**: A LangChain utility for loading PDF files into a format that can be used for further text processing.

5. **langchain.text_splitter.RecursiveCharacterTextSplitter**: A LangChain utility for splitting large text documents into smaller chunks based on characters or words, with some overlap for better context during embeddings.

6. **langchain.vectorstores.faiss.FAISS**: FAISS is a vector search engine used to store document embeddings and allows quick similarity-based retrieval.

7. **langchain_core.embeddings.Embeddings**: Base class for embedding models in LangChain.

8. **langchain.docstore.InMemoryDocstore**: Stores documents in memory for quick retrieval.

9. **faiss**: A fast library for nearest neighbor search in high-dimensional vector spaces, crucial for creating vector stores.

10. **numpy**: A scientific computing library used here for handling arrays and matrices, particularly embedding vectors.

11. **PyMuPDF** (also known as Fitz) is a Python binding for MuPDF, a lightweight PDF and XPS viewer. It allows users to extract text, images, and metadata from PDF files, as well as manipulate and annotate them

12. **PyMuPDF**
sklearn.metrics.pairwise is a module in scikit-learn that provides functions for evaluating pairwise distances or similarities between samples. It includes various distance metrics like cosine similarity, retrieval, and machine learning tasks.

In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain_core.embeddings import Embeddings
from langchain.docstore import InMemoryDocstore
import faiss
import numpy as np



In [ ]:
load_dotenv()
#add your
# TODO: Load the Azure OpenAI configuration from environment variables
# Hint: Use os.getenv() to get the API key, API version, and endpoint, feed the file to load_dotenv()
load_dotenv()
azure_openai_api_key = # Your code here
azure_openai_api_version = # Your code here
azure_endpoint = # Your code here

In [ ]:
# TODO: Print the configuration to verify it's loaded correctly
print()
# Your code here
# Your code here

## Custom Embeddings Class:

- This custom class **`CustomAzureEmbeddings`** is used to embed documents and queries using Azure OpenAI's embedding model.

### Class Structure:
- **`__init__` Method**: 
   - Initializes the class and sets up the Azure OpenAI client by providing an API key, API version, and the Azure endpoint.
   - The class interacts with Azure OpenAI's embeddings service to generate document embeddings.
  
- **`embed_documents` Method**: 
   - Takes a list of texts as input and returns embeddings for each text.
  
- **`embed_query` Method**:
   - Embeds a single text (query) by calling Azure OpenAI’s `embeddings.create` method.
   - Uses the `text-embedding-ada-002` model to generate embeddings.


In [ ]:
# Cell 4: Custom Embeddings Class

class CustomAzureEmbeddings(Embeddings):
    def __init__(self, ------, ------, ------):
        # TODO: Initialize the Azure OpenAI client
        # Hint: Use the AzureOpenAI class
        self.client = # Your code here

    def embed_documents(self, ------):
        # TODO: Implement the method to embed multiple documents
        # Hint: Use a list comprehension with self.embed_query
        return # Your code here

    def embed_query(self, ------):
        # TODO: Implement the method to embed a single query
        # Hint: Use self.client.embeddings.create(), use model="text-embedding-ada-002"
        response = # Your code here
        return response.data[0].embedding

## Encode PDF and Create FAISS Vector Store:

This function loads a PDF document, processes it into smaller chunks, embeds the chunks, and stores the embeddings in a FAISS index for efficient document retrieval.

### Detailed Steps:
1. **Azure OpenAI Configuration**:
   - Defines API keys, version, and the endpoint.
   
2. **CustomAzureEmbeddings Initialization**:
   - Initializes the `CustomAzureEmbeddings` class for interacting with Azure OpenAI’s embedding model.

3. **Load and Process PDF**:
   - Loads the PDF document using `PyPDFLoader`.
   - Splits the document into smaller chunks (with overlapping sections) using `RecursiveCharacterTextSplitter`.

4. **Generate Embeddings**:
   - Embeds the text chunks using the custom embeddings class.
   - Converts embeddings into a NumPy array for further processing.

5. **Create FAISS Index**:
   - Initializes a FAISS index with the dimensionality of the embeddings.
   - Adds the embeddings to the index.

6. **In-Memory Document Store**:
   - Uses `InMemoryDocstore` to store the original documents and maps the document embeddings in the FAISS index to the corresponding document IDs.

7. **Return Vector Store**:
   - Returns the FAISS vector store, which can now be used for retrieval based on similarity.


In [ ]:
# Cell 5: Encode PDF Function

def encode_pdf(path, chunk_size=300, chunk_overlap=200):
    try:
        # Initialize the custom Azure OpenAI embeddings class
        embeddings_client = CustomAzureEmbeddings(
            api_key=azure_openai_api_key,
            api_version=azure_openai_api_version,
            azure_endpoint=azure_endpoint
        )

        # TODO: Load and process the PDF
        # Hint: Use PyPDFLoader and RecursiveCharacterTextSplitter
        loader = # Your code here
        documents = # Your code here
        text_splitter = # Your code here
        texts = # Your code here
        text_list = [doc.page_content for doc in texts]

        # Generate embeddings
        embeddings = embeddings_client.embed_documents(text_list)
        embeddings_array = np.array(embeddings)
        print(f"Document embeddings shape: {embeddings_array.shape}")

        # TODO: Create FAISS index
        # Hint: Use faiss.IndexFlatL2
        dimension = embeddings_array.shape[1]
        index = # Your code here
        index.add(embeddings_array)

        print(f"FAISS index dimension: {index.d}")
        print(f"Number of vectors in FAISS index: {index.ntotal}")

        # Create InMemoryDocstore and index mapping
        docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(texts)})
        index_to_docstore_id = {i: str(i) for i in range(len(texts))}

        # TODO: Create FAISS vector store
        # Hint: Use the FAISS class from langchain.vectorstores
        vectorstore = # Your code here

        return vectorstore

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

## Main Execution for Vector Store Creation:

This section defines the path to the PDF file and calls the `encode_pdf` function to create a FAISS vector store from the PDF content. If any errors occur during the process, they are caught and printed.


In [ ]:
# Cell 6: Create Vector Store

# Path to your PDF
pdf_path = './data/Understanding_Climate_Change.pdf'

# TODO: Encode the PDF and create the vector store
# Hint: Use the encode_pdf function and handle exceptions
try:
    vectorstore = # Your code here
    print("Vector store created successfully.")
except Exception as e:
    print(f"An error occurred during vector store creation: {e}")

## Document Retrieval:

This part uses the FAISS vector store to retrieve the most relevant documents based on the user’s query.

### Steps:
1. **Create a Retriever**: 
   - A retriever is created from the vector store using `as_retriever()`.
   - `search_kwargs={"k": 5}` specifies that the top 5 relevant documents should be retrieved.

2. **Define Query**: 
   - The query asks about the main cause of climate change.

3. **Retrieve Documents**:
   - The retriever finds the top 5 relevant documents and concatenates their content into a `context`.
   - If an error occurs during retrieval, it is caught and printed.


In [ ]:
# Cell 7: Document Retrieval

# TODO: Create a retriever from the vector store
# Hint: Use the as_retriever() method
retriever = # Your code here

# Define a query
query = "What is the main cause of climate change?"

# TODO: Retrieve relevant documents
# Hint: Use the get_relevant_documents() method and join the results
try:
    docs = # Your code here
    # TODO: Create context by joining the content of retrieved documents
    # Hint: Use a list comprehension to extract page_content and join with "\n\n"
    context = # Your code here
    print("\nRetrieved context:")
    print(context)
except Exception as e:
    print(f"An error occurred during document retrieval: {e}")





## Generate Answer Using Azure OpenAI:

This section calls Azure OpenAI’s `GPT-4o` model to generate an answer to the user’s query based on the retrieved document context.

### Steps:
1. **Initialize Azure OpenAI Chat Client**: 
   - The chat client is initialized using the Azure OpenAI API key, version, and endpoint.

2. **Create Chat Completion Request**: 
   - A completion request is made to Azure OpenAI, providing it with the retrieved context and user’s query.
   - The system message sets the assistant's behavior as helpful.

3. **Generate Answer**:
   - The `gpt-4o` model generates an answer to the query based on the context.
   - If an error occurs, it is caught and printed.


In [ ]:
# Cell 8: Generate Answer Using Azure OpenAI

# TODO: Initialize the Azure OpenAI chat client
# Hint: Use the AzureOpenAI class
chat_client = # Your code here

# TODO: Generate an answer using the chat model
# Hint: Use chat_client.chat.completions.create()
try:
    response = # Your code here
    print("\nAnswer:")
    # Hint: Access the content of the first choice's message in the response
    print(# Your code here)
except Exception as e:
    print(f"An error occurred during answer generation: {e}")
